<a href="https://colab.research.google.com/github/chrismarkella/Kaggle-access-from-Google-Colab/blob/master/sql_pandas_nobel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

import numpy as np
import pandas as pd

from getpass import getpass 

In [2]:
def access_kaggle():
    """
    Access Kaggle from Google Colab.
    If the /root/.kaggle does not exist then prompt for
    the username and for the Kaggle API key.
    Creates the kaggle.json access file in the /root/.kaggle/ folder. 
    """
    KAGGLE_ROOT = os.path.join('/root', '.kaggle')
    KAGGLE_PATH = os.path.join(KAGGLE_ROOT, 'kaggle.json')

    if '.kaggle' not in os.listdir(path='/root'):
        user = getpass(prompt='Kaggle username: ')
        key  = getpass(prompt='Kaggle API key: ')
        
        !mkdir $KAGGLE_ROOT
        !touch $KAGGLE_PATH
        !chmod 666 $KAGGLE_PATH
        with open(KAGGLE_PATH, mode='w') as f:
            f.write('{"username":"%s", "key":"%s"}' %(user, key))
            f.close()
        !chmod 600 $KAGGLE_PATH
        del user
        del key
        success_msg = "Kaggle is successfully set up. Good to go."
        print(f'{success_msg}')

access_kaggle()


Kaggle username: ··········
Kaggle API key: ··········
Kaggle is successfully set up. Good to go.


In [3]:
!kaggle datasets list -s nobel

ref                                        title                                 size  lastUpdated          downloadCount  
-----------------------------------------  -----------------------------------  -----  -------------------  -------------  
nobelfoundation/nobel-laureates            Nobel Laureates, 1901-Present         66KB  2017-02-16 00:31:00           2503  
mbogernetto/women-in-nobel-prize-19012019  Women in Nobel Prize (1901-2019)      39KB  2020-01-08 17:08:24             24  
nayansolanki2411/nobel-prize-19012017      Nobel prize (1901-2017)               35KB  2018-05-30 16:29:13            120  
mpwolke/cusersmarildownloadsdeathscsv      Mortality among children               8KB  2019-10-17 23:48:09            112  
cdc/mortality                              Death in the United States           731MB  2017-08-03 19:10:09          15145  
eurostat/european-union                    Member States of the European Union    2KB  2017-03-14 18:12:13            670  
vanshjat

In [4]:
!kaggle datasets files nobelfoundation/nobel-laureates

name          size  creationDate         
-----------  -----  -------------------  
archive.csv  283KB  2017-02-16 00:31:00  


In [6]:
!kaggle datasets download nobelfoundation/nobel-laureates --unzip -p ./datasets
!ls -lh ./datasets

  0% 0.00/66.2k [00:00<?, ?B/s]
100% 66.2k/66.2k [00:00<00:00, 62.9MB/s]
total 284K
-rw-r--r-- 1 root root 284K Feb  3 04:46 archive.csv


In [7]:
df = pd.read_csv('datasets/archive.csv', sep=',')
df.columns

Index(['Year', 'Category', 'Prize', 'Motivation', 'Prize Share', 'Laureate ID',
       'Laureate Type', 'Full Name', 'Birth Date', 'Birth City',
       'Birth Country', 'Sex', 'Organization Name', 'Organization City',
       'Organization Country', 'Death Date', 'Death City', 'Death Country'],
      dtype='object')

In [10]:
columns_to_load = [
    'Year',
    'Category',
    'Full Name',
]
df = pd.read_csv('datasets/archive.csv', sep=',',
                 usecols=columns_to_load)
df.head(3)

,Year,Category,Full Name
0,1901,Chemistry,Jacobus Henricus van 't Hoff
1,1901,Literature,Sully Prudhomme
2,1901,Medicine,Emil Adolf von Behring


In [17]:
column_name_dict = {
    'Year': 'yr',
    'Category': 'subject',
    'Full Name':'winner'
}
df.rename(mapper=column_name_dict, axis='columns',
          inplace=True)
df.head(3)

,yr,subject,winner
0,1901,Chemistry,Jacobus Henricus van 't Hoff
1,1901,Literature,Sully Prudhomme
2,1901,Medicine,Emil Adolf von Behring




```sql
SELECT yr, subject, winner
FROM nobel
WHERE yr = 1950
```



In [19]:
filt_year = (df.yr == 1950)
selected_columns = [
    'yr',
    'subject',
    'winner',
]

df.loc[filt_year, selected_columns]

,yr,subject,winner
259,1950,Chemistry,Otto Paul Hermann Diels
260,1950,Chemistry,Kurt Alder
261,1950,Literature,Earl (Bertrand Arthur William) Russell
262,1950,Medicine,Edward Calvin Kendall
263,1950,Medicine,Tadeus Reichstein
264,1950,Medicine,Philip Showalter Hench
265,1950,Peace,Ralph Bunche
266,1950,Physics,Cecil Frank Powell




```sql
SELECT winner
FROM nobel
WHERE yr = 1962 AND 
      subject = 'Literature'
```



In [20]:
filt_year = (df.yr == 1962)
filt_subject = (df.subject == 'Literature')
filt = (filt_year & filt_subject)

df.loc[filt, 'winner']

353    John Steinbeck
Name: winner, dtype: object



```sql
SELECT yr, subject
FROM nobel
WHERE winner = 'Albert Einstein'
```



In [21]:
filt_winner = (df.winner == 'Albert Einstein')

df.loc[filt_winner, ['yr', 'subject']]

,yr,subject
107,1921,Physics




```sql
SELECT winner
FROM nobel
WHERE subject = 'Peace' AND
      yr >= 2000
```



In [22]:
filt_subject = (df.subject == 'Peace')
filt_year = (df.yr >= 2000)
filt = (filt_subject & filt_year)

df.loc[filt, 'winner']

740                                         Kim Dae-jung
754                                United Nations (U.N.)
755                                           Kofi Annan
769                                         Jimmy Carter
781                                         Shirin Ebadi
795                                 Wangari Muta Maathai
808            International Atomic Energy Agency (IAEA)
809                                    Mohamed ElBaradei
820                                       Muhammad Yunus
821                                         Grameen Bank
833     Intergovernmental Panel on Climate Change (IPCC)
834                          Albert Arnold (Al) Gore Jr.
848                                     Martti Ahtisaari
866                                      Barack H. Obama
880                                           Liu Xiaobo
891                                Ellen Johnson Sirleaf
892                                        Leymah Gbowee
893                            



```sql
SELECT *
FROM nobel
WHERE subject = 'Literature' AND yr  BETWEEN 1980 AND 1989
```



In [23]:
filt_year = (df.yr.between(1980, 1989))
filt_subj = (df.subject == 'Literature')

filt = (filt_year & filt_subj)

df.loc[filt]

,yr,subject,winner
529,1980,Literature,Czeslaw Milosz
539,1981,Literature,Elias Canetti
549,1982,Literature,Gabriel García Márquez
558,1983,Literature,William Golding
565,1984,Literature,Jaroslav Seifert
575,1985,Literature,Claude Simon
584,1986,Literature,Wole Soyinka
596,1987,Literature,Joseph Brodsky
606,1988,Literature,Naguib Mahfouz
617,1989,Literature,Camilo José Cela




```sql
SELECT * FROM nobel
WHERE winner IN ('Theodore Roosevelt','Woodrow Wilson',
                 'Jimmy Carter', 'Barack Obama')
```



In [25]:
president_lst = [
    'Theodore Roosevelt',
    'Woodrow Wilson',
    'Jimmy Carter',
    'Barack H. Obama',
]
filt_winner = (df.winner.isin(president_lst))

df.loc[filt_winner]

,yr,subject,winner
35,1906,Peace,Theodore Roosevelt
769,2002,Peace,Jimmy Carter
866,2009,Peace,Barack H. Obama
